In [2]:
# Setting up the environment

import pandas as pd
from dotenv import load_dotenv
import os 

load_dotenv()

df: pd.Series = pd.read_json('db.json',typ='series')

df = df[:5]

from utils.html_tools import extract_text

df = df.apply(lambda x: extract_text(x))
df

index                                    Angular Documentation Angular is an applicati...
guide/what-is-angular                    What is Angular? This topic can help you unde...
start/start-routing                      Adding navigation This guide builds on the fi...
start/start-forms                        Using forms for user input This guide builds ...
guide/setup-local                        Setting up the local environment and workspac...
guide/understanding-angular-overview     Understanding Angular To understand the capab...
guide/component-overview                 Angular components overview Components are th...
guide/elements                           Angular elements overview Angular elements ar...
start                                    Getting started with Angular Welcome to Angul...
start/start-data                         Managing data This guide builds on the second...
dtype: object

In [5]:
from utils.open_ai_tools import parse_to_blink
import openai
# OpenAI Integration
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "OOPS, please set env var called 'OPEN_AI_API_KEY'")

In [ ]:
# Langchain
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=OPENAI_API_KEY, max_tokens=-1,model_name="text-davinci-003")
from langchain import PromptTemplate

template = """
You are a programming teacher.
Avoid newline characters outside of code blocks.
Generate a concise and informative 'blink' from the provided HTML content, adhering to this JSON structure: 
(
"title": "<title>",
"description": "<description>",
"code": "<code>", # optional, up to 15 lines, leave empty if not applicable
"difficulty": "<beginner | intermediate | advanced>"
)

HTML content to convert into a blink: {html}
"""

prompt = PromptTemplate(
    input_variables=["html"],
    template=template,
)
from utils.open_ai_tools import truncate_text

blinks = {}
for index, i in enumerate(df.values):
    parsed_text = truncate_text(i, 1800)
    blink_prompt = prompt.format(html=parsed_text)
    blinks[index] = llm.predict(blink_prompt)

In [ ]:
blinks